In [1]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim import matutils, corpora
from gensim.models.ldamodel import LdaModel
import pandas as pd
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.gensim
import numpy as np 
import snowball

In [4]:
###
# load our corpus with Pandas; this is a tab-separate file of tweets
tweet_data = pd.read_csv("CancerReport-clean-whitelisted-en.txt", delimiter="\t")

In [5]:
tweet_data.shape
#dir(snowball)

(13345, 40)

In [6]:
tweet_data.head()

,tweet_id,tweet_text,tweet_created_at,in_reply_to_status_id_str,in_reply_to_screen_name,retweet_count,favorite_count,machine_translated_language,geo_lat,geo_long,...,retweet_created_at,retweet_screen_name,retweet_user_created_at,retweet_person_name,retweet_statuses_count,retweet_friends_count,retweet_followers_count,retweet_urls,retweet_hashtag_text,retweet_usermentions_screen_name
0,185081180222263297,My arm is so sore :( had a HPV injection at sc...,2012-03-28 15:09:26,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185096288109461507,MsRhea Mike HEALTHY Wednesday Awareness mak...,2012-03-28 16:09:28,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,185108334121193472,@xomorgyyxo I asked him three times if it was ...,2012-03-28 16:57:20,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185070816092880896,Just informed by a 26 year old Human papillom...,2012-03-28 14:28:15,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185173819793735680,"If someone tells you that ""everybody"" has HPV ...",2012-03-28 21:17:33,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
primary_tweets = tweet_data[tweet_data["retweet"] == False]

In [8]:
tweets = tweet_data["tweet_text"].tolist()

In [9]:
tweets[0]

"My arm is so sore :( had a HPV injection at school today :'( xxx"

In [10]:
# this will be *all* tweets; independent of the hashtag
toked_tweets, kept_indices = snowball.build_gensim_corpus(tweets)

In [11]:
print(list(toked_tweets[0]))

['arm', 'sore', 'hpv', 'injection', 'school', 'today', 'xxx']


In [15]:
dictionary = corpora.Dictionary(toked_tweets)
print(dictionary)

In [21]:
gensim_corpus = [dictionary.doc2bow(tweet) for tweet in toked_tweets]

In [20]:
# this takes quite some time!
lda = LdaModel(gensim_corpus, num_topics=10,
               passes=10, alpha=0.001, 
               id2word=dictionary)

In [22]:
lda.show_topics()

['0.046*rt + 0.036*amp + 0.024*women + 0.023*planned + 0.022*parenthood + 0.022*breast + 0.014*screenings + 0.013*control + 0.011*hiv + 0.010*died',
 '0.107*http + 0.103*co + 0.101*hpv + 0.069*vaccine + 0.039*rt + 0.027*girls + 0.027*gardasil + 0.021*vaccines + 0.020*boys + 0.016*vaccination',
 '0.153*cancer + 0.139*cervical + 0.093*http + 0.088*co + 0.046*rt + 0.017*women + 0.017*screening + 0.010*prevent + 0.010*awareness + 0.009*amp',
 '0.031*vax + 0.030*hpv + 0.029*dr + 0.023*exams + 0.014*low + 0.013*mother + 0.012*must + 0.012*see + 0.011*strains + 0.011*affordable',
 '0.048*http + 0.047*co + 0.027*hpv + 0.017*amp + 0.012*lead + 0.012*rates + 0.011*dangerous + 0.011*hiv + 0.011*review + 0.010*deadly',
 '0.101*co + 0.093*http + 0.052*gardasil + 0.041*rt + 0.040*gt + 0.025*hpv + 0.022*vaccine + 0.016*https + 0.010*via + 0.006*de',
 '0.044*cancer + 0.042*hpv + 0.041*get + 0.041*cervical + 0.031*shot + 0.026*rt + 0.020*got + 0.014*please + 0.012*today + 0.012*age',
 '0.136*hpv + 0.04

In [23]:
for_viz = pyLDAvis.gensim.prepare(lda, gensim_corpus, dictionary)

In [24]:
pyLDAvis.display(for_viz)